In [3]:
import re
import ast
import pickle
import numpy as np 
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler

import nltk
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

import warnings
warnings.filterwarnings('ignore')

# Data collection

In [5]:
songs = pd.read_csv('dataset.csv')
songs

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,...,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.7150,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,...,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.2670,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,...,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.1200,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.1430,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,...,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.1670,119.949,4,acoustic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113995,113995,2C3TZjDRiAzdyViavDJ217,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Sleep My Little Boy,21,384999,False,0.172,0.2350,...,-16.393,1,0.0422,0.6400,0.928000,0.0863,0.0339,125.995,5,world-music
113996,113996,1hIz5L4IB9hN3WRYPOCGPw,Rainy Lullaby,#mindfulness - Soft Rain for Mindful Meditatio...,Water Into Light,22,385000,False,0.174,0.1170,...,-18.318,0,0.0401,0.9940,0.976000,0.1050,0.0350,85.239,4,world-music
113997,113997,6x8ZfSoqDjuNa5SVP5QjvX,Cesária Evora,Best Of,Miss Perfumado,22,271466,False,0.629,0.3290,...,-10.895,0,0.0420,0.8670,0.000000,0.0839,0.7430,132.378,4,world-music
113998,113998,2e6sXL2bYv4bSz6VTdnfLs,Michael W. Smith,Change Your World,Friends,41,283893,False,0.587,0.5060,...,-10.889,1,0.0297,0.3810,0.000000,0.2700,0.4130,135.960,4,world-music


In [8]:
songs = songs[:5000]

# Data cleaning

In [11]:
songs.nunique()

Unnamed: 0          5000
track_id            4404
artists             1450
album_name          2438
track_name          3753
popularity            87
duration_ms         3725
explicit               2
danceability         777
energy              1163
key                   12
loudness            3470
mode                   2
speechiness          827
acousticness        1765
instrumentalness    1834
liveness             969
valence             1183
tempo               3788
time_signature         5
track_genre            5
dtype: int64

In [13]:
songs.columns

Index(['Unnamed: 0', 'track_id', 'artists', 'album_name', 'track_name',
       'popularity', 'duration_ms', 'explicit', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature',
       'track_genre'],
      dtype='object')

In [15]:
songs['liveness']

0       0.3580
1       0.1010
2       0.1170
3       0.1320
4       0.0829
         ...  
4995    0.1030
4996    0.1110
4997    0.0945
4998    0.1170
4999    0.1060
Name: liveness, Length: 5000, dtype: float64

In [17]:
songs['danceability']

0       0.6760
1       0.4200
2       0.4380
3       0.2660
4       0.6180
         ...  
4995    0.4650
4996    0.5160
4997    0.2220
4998    0.0576
4999    0.1560
Name: danceability, Length: 5000, dtype: float64

In [19]:
songs = songs[['track_id', 'artists', 'album_name', 'track_name','danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness','instrumentalness', 'liveness', 'valence', 'tempo','track_genre']]
songs

,track_id,artists,album_name,track_name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,track_genre
0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,0.6760,0.4610,1,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.7150,87.917,acoustic
1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,0.4200,0.1660,1,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.2670,77.489,acoustic
2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,0.4380,0.3590,0,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.1200,76.332,acoustic
3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,0.2660,0.0596,0,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.1430,181.740,acoustic
4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,0.6180,0.4430,2,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.1670,119.949,acoustic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,5i4zyaQRXAX9oI2C9dmHdB,Ludovico Einaudi,Elements (Deluxe),Drop,0.4650,0.0342,2,-28.257,0,0.0401,0.9920,0.939000,0.1030,0.0590,152.054,ambient
4996,5tR13oIwi5J3elcTZPTTIU,Air,Moon Safari,Kelly Watch the Stars,0.5160,0.7430,2,-8.470,1,0.0292,0.2820,0.637000,0.1110,0.2260,109.837,ambient
4997,76R8ILULQyyodS91TkAvOf,Ludovico Einaudi,Islands - Essential Einaudi,Passagio,0.2220,0.0200,9,-27.114,1,0.0464,0.9950,0.948000,0.0945,0.1190,85.860,ambient
4998,1GyODtIqjM5AO3z5dEJwRg,Liquid Mind,Liquid Mind II: Slow World,Blue Seven,0.0576,0.1690,10,-20.216,1,0.0484,0.9030,0.966000,0.1170,0.0310,50.838,ambient


In [21]:
songs.isnull().sum()

track_id            0
artists             0
album_name          0
track_name          0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
track_genre         0
dtype: int64

In [23]:
songs.dropna(axis = 0, how = 'any' , inplace = True)

In [25]:
songs.head()

,track_id,artists,album_name,track_name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,track_genre
0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,0.676,0.4610,1,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,acoustic
1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,0.420,0.1660,1,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,acoustic
2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,0.438,0.3590,0,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,acoustic
3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,0.266,0.0596,0,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,acoustic
4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,0.618,0.4430,2,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,acoustic


In [27]:
songs['mode'].unique()

array([0, 1], dtype=int64)

In [29]:
songs.isnull().sum()

track_id            0
artists             0
album_name          0
track_name          0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
track_genre         0
dtype: int64

# Split columns into text based and number based 

## numerical

In [34]:
num = songs[['danceability', 'energy','key', 'loudness', 'mode', 'speechiness', 'acousticness','instrumentalness', 'liveness', 'valence', 'tempo']]
num

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.6760,0.4610,1,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.7150,87.917
1,0.4200,0.1660,1,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.2670,77.489
2,0.4380,0.3590,0,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.1200,76.332
3,0.2660,0.0596,0,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.1430,181.740
4,0.6180,0.4430,2,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.1670,119.949
...,...,...,...,...,...,...,...,...,...,...,...
4995,0.4650,0.0342,2,-28.257,0,0.0401,0.9920,0.939000,0.1030,0.0590,152.054
4996,0.5160,0.7430,2,-8.470,1,0.0292,0.2820,0.637000,0.1110,0.2260,109.837
4997,0.2220,0.0200,9,-27.114,1,0.0464,0.9950,0.948000,0.0945,0.1190,85.860
4998,0.0576,0.1690,10,-20.216,1,0.0484,0.9030,0.966000,0.1170,0.0310,50.838


In [36]:
sc = StandardScaler()
sc_num = sc.fit_transform(num[:])
sc_num

array([[ 0.82403662, -0.38490309, -1.2281132 , ...,  1.20050913,
         0.94058014, -1.04481136],
       [-0.68592211, -1.4274839 , -1.2281132 , ..., -0.49238692,
        -0.71705252, -1.39263954],
       [-0.57975314, -0.74538865, -1.50919742, ..., -0.38699261,
        -1.26096323, -1.43123153],
       ...,
       [-1.85378081, -1.94347304,  1.02056061, ..., -0.53520336,
        -1.26466331, -1.11342303],
       [-2.82345744, -1.41688138,  1.30164483, ..., -0.38699261,
        -1.59026972, -2.2815894 ],
       [-2.24306705, -1.56885079, -1.50919742, ..., -0.4594512 ,
        -1.57879949, -1.4831656 ]])

In [38]:
num_sim = cosine_similarity(sc_num)
num_sim.shape

(5000, 5000)

# textual

In [41]:
text = songs[['track_id', 'artists', 'album_name', 'track_name','track_genre']]
text

,track_id,artists,album_name,track_name,track_genre
0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,acoustic
1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,acoustic
2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,acoustic
3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,acoustic
4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,acoustic
...,...,...,...,...,...
4995,5i4zyaQRXAX9oI2C9dmHdB,Ludovico Einaudi,Elements (Deluxe),Drop,ambient
4996,5tR13oIwi5J3elcTZPTTIU,Air,Moon Safari,Kelly Watch the Stars,ambient
4997,76R8ILULQyyodS91TkAvOf,Ludovico Einaudi,Islands - Essential Einaudi,Passagio,ambient
4998,1GyODtIqjM5AO3z5dEJwRg,Liquid Mind,Liquid Mind II: Slow World,Blue Seven,ambient


In [43]:
text['tags'] = text.apply(lambda row: [row['artists'], row['album_name'], row['track_genre']], axis=1)
text['tags']

0                         [Gen Hoshino, Comedy, acoustic]
1              [Ben Woodward, Ghost (Acoustic), acoustic]
2       [Ingrid Michaelson;ZAYN, To Begin Again, acous...
3       [Kina Grannis, Crazy Rich Asians (Original Mot...
4                   [Chord Overstreet, Hold On, acoustic]
                              ...                        
4995       [Ludovico Einaudi, Elements (Deluxe), ambient]
4996                          [Air, Moon Safari, ambient]
4997    [Ludovico Einaudi, Islands - Essential Einaudi...
4998    [Liquid Mind, Liquid Mind II: Slow World, ambi...
4999    [Liquid Mind, Liquid Mind XI: Deep Sleep, ambi...
Name: tags, Length: 5000, dtype: object

In [45]:
text['tags'] = text['tags'].apply(lambda x : [i.replace(' ' , '') for i in x])

In [47]:
text

,track_id,artists,album_name,track_name,track_genre,tags
0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,acoustic,"[GenHoshino, Comedy, acoustic]"
1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,acoustic,"[BenWoodward, Ghost(Acoustic), acoustic]"
2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,acoustic,"[IngridMichaelson;ZAYN, ToBeginAgain, acoustic]"
3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,acoustic,"[KinaGrannis, CrazyRichAsians(OriginalMotionPi..."
4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,acoustic,"[ChordOverstreet, HoldOn, acoustic]"
...,...,...,...,...,...,...
4995,5i4zyaQRXAX9oI2C9dmHdB,Ludovico Einaudi,Elements (Deluxe),Drop,ambient,"[LudovicoEinaudi, Elements(Deluxe), ambient]"
4996,5tR13oIwi5J3elcTZPTTIU,Air,Moon Safari,Kelly Watch the Stars,ambient,"[Air, MoonSafari, ambient]"
4997,76R8ILULQyyodS91TkAvOf,Ludovico Einaudi,Islands - Essential Einaudi,Passagio,ambient,"[LudovicoEinaudi, Islands-EssentialEinaudi, am..."
4998,1GyODtIqjM5AO3z5dEJwRg,Liquid Mind,Liquid Mind II: Slow World,Blue Seven,ambient,"[LiquidMind, LiquidMindII:SlowWorld, ambient]"


In [49]:
text.drop(['artists','album_name', 'track_genre'] , axis =1,inplace =True)

In [51]:
text.head()

,track_id,track_name,tags
0,5SuOikwiRyPMVoIQDJUgSV,Comedy,"[GenHoshino, Comedy, acoustic]"
1,4qPNDBW1i3p13qLCt0Ki3A,Ghost - Acoustic,"[BenWoodward, Ghost(Acoustic), acoustic]"
2,1iJBSr7s7jYXzM8EGcbK5b,To Begin Again,"[IngridMichaelson;ZAYN, ToBeginAgain, acoustic]"
3,6lfxq3CG4xtTiEg7opyCyx,Can't Help Falling In Love,"[KinaGrannis, CrazyRichAsians(OriginalMotionPi..."
4,5vjLSffimiIP26QG5WcN2K,Hold On,"[ChordOverstreet, HoldOn, acoustic]"


In [53]:
text['tags'] = text['tags'].apply(lambda x : " ".join(x))

In [55]:
text.head()

,track_id,track_name,tags
0,5SuOikwiRyPMVoIQDJUgSV,Comedy,GenHoshino Comedy acoustic
1,4qPNDBW1i3p13qLCt0Ki3A,Ghost - Acoustic,BenWoodward Ghost(Acoustic) acoustic
2,1iJBSr7s7jYXzM8EGcbK5b,To Begin Again,IngridMichaelson;ZAYN ToBeginAgain acoustic
3,6lfxq3CG4xtTiEg7opyCyx,Can't Help Falling In Love,KinaGrannis CrazyRichAsians(OriginalMotionPict...
4,5vjLSffimiIP26QG5WcN2K,Hold On,ChordOverstreet HoldOn acoustic


In [57]:
text['tags'] = text['tags'].apply(lambda x : x.lower())

In [59]:
text.head()

,track_id,track_name,tags
0,5SuOikwiRyPMVoIQDJUgSV,Comedy,genhoshino comedy acoustic
1,4qPNDBW1i3p13qLCt0Ki3A,Ghost - Acoustic,benwoodward ghost(acoustic) acoustic
2,1iJBSr7s7jYXzM8EGcbK5b,To Begin Again,ingridmichaelson;zayn tobeginagain acoustic
3,6lfxq3CG4xtTiEg7opyCyx,Can't Help Falling In Love,kinagrannis crazyrichasians(originalmotionpict...
4,5vjLSffimiIP26QG5WcN2K,Hold On,chordoverstreet holdon acoustic


In [61]:
cv = CountVectorizer( stop_words='english')
vectors = cv.fit_transform(text['tags']).toarray()

In [63]:
vectors.shape

(5000, 4380)

In [65]:
text_sim = cosine_similarity(vectors)
text_sim.shape

(5000, 5000)

In [66]:
weight_num = 0.7
weight_text = 0.3

# Combine the matrices with the assigned weights
combined_sim = (weight_num * num_sim + weight_text * text_sim)
combined_sim

array([[ 1.        ,  0.11567458,  0.17343711, ..., -0.32766897,
        -0.29255265, -0.20708247],
       [ 0.11567458,  1.        ,  0.60867446, ...,  0.42285818,
         0.38964688,  0.52242688],
       [ 0.17343711,  0.60867446,  1.        , ...,  0.1431413 ,
         0.21899617,  0.36718221],
       ...,
       [-0.32766897,  0.42285818,  0.1431413 , ...,  1.        ,
         0.725743  ,  0.67456085],
       [-0.29255265,  0.38964688,  0.21899617, ...,  0.725743  ,
         1.        ,  0.73746592],
       [-0.20708247,  0.52242688,  0.36718221, ...,  0.67456085,
         0.73746592,  1.        ]])

In [67]:
sorted(enumerate(combined_sim[3]) , reverse = True , key = lambda x : x[1])[1:11]
# enumerate coverts every pair into pair of position , value so the key specefies sorting should be based on 1 index that is similarity

[(279, 0.8398797757190555),
 (211, 0.7616924020350071),
 (952, 0.7610826936272311),
 (607, 0.7610154685235373),
 (354, 0.7598357158550956),
 (415, 0.7583543474919107),
 (35, 0.7495816710459604),
 (58, 0.7461652677530509),
 (206, 0.742802037971118),
 (33, 0.7416637103709541)]

In [68]:
songs.iloc[211]

track_id            5eNYumDY5PCXtKKJ0SWg2r
artists                     The Civil Wars
album_name                   Barton Hollow
track_name                   Poison & Wine
danceability                         0.285
energy                               0.186
key                                      1
loudness                           -12.729
mode                                     1
speechiness                         0.0288
acousticness                          0.76
instrumentalness                  0.000073
liveness                             0.104
valence                              0.203
tempo                              153.848
track_genre                       acoustic
Name: 211, dtype: object

# Final Function

In [70]:
songs.head(10)

,track_id,artists,album_name,track_name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,track_genre
0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,0.676,0.4610,1,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.7150,87.917,acoustic
1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,0.420,0.1660,1,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.2670,77.489,acoustic
2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,0.438,0.3590,0,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.1200,76.332,acoustic
3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,0.266,0.0596,0,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.1430,181.740,acoustic
4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,0.618,0.4430,2,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.1670,119.949,acoustic
5,01MVOl9KtVTNfFiBU9I7dc,Tyrone Wells,Days I Will Remember,Days I Will Remember,0.688,0.4810,6,-8.807,1,0.1050,0.2890,0.000000,0.1890,0.6660,98.017,acoustic
6,6Vc5wAMmXdKIAM7WUoEb7N,A Great Big World;Christina Aguilera,Is There Anybody Out There?,Say Something,0.407,0.1470,2,-8.822,1,0.0355,0.8570,0.000003,0.0913,0.0765,141.284,acoustic
7,1EzrEOXmMH3G43AXT1y7pA,Jason Mraz,We Sing. We Dance. We Steal Things.,I'm Yours,0.703,0.4440,11,-9.331,1,0.0417,0.5590,0.000000,0.0973,0.7120,150.960,acoustic
8,0IktbUcnAGrvD03AWnz3Q8,Jason Mraz;Colbie Caillat,We Sing. We Dance. We Steal Things.,Lucky,0.625,0.4140,0,-8.700,1,0.0369,0.2940,0.000000,0.1510,0.6690,130.088,acoustic
9,7k9GuJYLp2AzqokyEdwEw2,Ross Copperman,Hunger,Hunger,0.442,0.6320,1,-6.770,1,0.0295,0.4260,0.004190,0.0735,0.1960,78.899,acoustic


In [143]:
def song_rec(text):
    recommended_songs = []
    track_id =[]
    index = songs[songs['track_name']== text].index[0]
    distance = combined_sim[index]
    song_lst = sorted(list(enumerate(distance)) , reverse = True , key = lambda x : x[1])[1:11]
    
    for i in song_lst:
        # print(i[0])
        recommended_songs.append((songs.iloc[i[0]].track_name))
        track_id.append(songs.iloc[i[0]].track_id)
    return recommended_songs 

In [145]:
type(song_rec('Ghost - Acoustic'))

list

In [147]:
song_rec('Hold On')

['Hold On',
 'Hold On - Acoustic',
 "What's Left of You",
 'Love You To Death',
 'Wind in Your Sails',
 'Tortured Soul',
 "She's The Song - Acoustic",
 'Use Somebody - Acoustic',
 'Stay Right Where You Are',
 'Without Me']

In [149]:
song_rec('Days I Will Remember')

['Days I Will Remember',
 'Pop Virus',
 '接吻',
 'Corinna - From "The Natch\'l Blues"',
 'Look For The Good (Single Version)',
 'DBS Out',
 'On & On',
 'La Dormida',
 'Original Joy',
 'Memories (Acoustic)']

In [75]:
import pickle

In [76]:
songs

,track_id,artists,album_name,track_name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,track_genre
0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,0.6760,0.4610,1,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.7150,87.917,acoustic
1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,0.4200,0.1660,1,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.2670,77.489,acoustic
2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,0.4380,0.3590,0,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.1200,76.332,acoustic
3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,0.2660,0.0596,0,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.1430,181.740,acoustic
4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,0.6180,0.4430,2,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.1670,119.949,acoustic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,5i4zyaQRXAX9oI2C9dmHdB,Ludovico Einaudi,Elements (Deluxe),Drop,0.4650,0.0342,2,-28.257,0,0.0401,0.9920,0.939000,0.1030,0.0590,152.054,ambient
4996,5tR13oIwi5J3elcTZPTTIU,Air,Moon Safari,Kelly Watch the Stars,0.5160,0.7430,2,-8.470,1,0.0292,0.2820,0.637000,0.1110,0.2260,109.837,ambient
4997,76R8ILULQyyodS91TkAvOf,Ludovico Einaudi,Islands - Essential Einaudi,Passagio,0.2220,0.0200,9,-27.114,1,0.0464,0.9950,0.948000,0.0945,0.1190,85.860,ambient
4998,1GyODtIqjM5AO3z5dEJwRg,Liquid Mind,Liquid Mind II: Slow World,Blue Seven,0.0576,0.1690,10,-20.216,1,0.0484,0.9030,0.966000,0.1170,0.0310,50.838,ambient


In [77]:
combined_sim

array([[ 1.        ,  0.11567458,  0.17343711, ..., -0.32766897,
        -0.29255265, -0.20708247],
       [ 0.11567458,  1.        ,  0.60867446, ...,  0.42285818,
         0.38964688,  0.52242688],
       [ 0.17343711,  0.60867446,  1.        , ...,  0.1431413 ,
         0.21899617,  0.36718221],
       ...,
       [-0.32766897,  0.42285818,  0.1431413 , ...,  1.        ,
         0.725743  ,  0.67456085],
       [-0.29255265,  0.38964688,  0.21899617, ...,  0.725743  ,
         1.        ,  0.73746592],
       [-0.20708247,  0.52242688,  0.36718221, ...,  0.67456085,
         0.73746592,  1.        ]])

In [78]:
pickle.dump(songs.to_dict() , open('songs.pkl' , mode = 'wb'))

In [288]:
pickle.dump(combined_sim , open('combined_sim.pkl' , mode = 'wb'))

In [153]:
import lzma
import pickle

# Saving with lzma compression
with lzma.open('combined_sim.pkl.xz', 'wb') as f:
    pickle.dump(combined_sim, f)



In [155]:
import bz2
import pickle

# Saving with bz2 compression
with bz2.open('combined_sim.pkl.bz2', 'wb') as f:
    pickle.dump(combined_sim, f)

#
